### JAEN 모듈 설치

In [1]:
!pip install jaen

### JAEN 모듈 import
- 대문자로 입력

In [1]:
import JAEN
from JAEN import list_data, download_data, Project

### 개인정보 수정
- 이메일 정보 수정, 나머지는 수정하지 말 것!

In [2]:
project_name = "손글씨 이미지 분류" # 프로젝트 이름(수정하지 마세요)
email = "scsa2407" # 이메일(본인 SCSA 이메일 입력)

class_info = {
    'edu_name':'SCSA', # 과정명
    'edu_rnd':'24기', # 차수
    'edu_class':'SW' # "없음"이면 빈 문자열
}

# 프로젝트 생성
pjt = Project(project_name=project_name, class_info=class_info, email=email)

<br/>
<br/>

### 필요 모듈 import 및 기본 설정

In [3]:
import pandas as pd
import numpy as np
import warnings


# 경고 무시
warnings.filterwarnings('ignore')

SEED=123
np.random.seed(SEED)

### 데이터 다운로드 및 제출 점검

In [4]:
download_data(project_name)

파일 다운로드 완료

데이터셋: 손글씨 이미지 분류
파일경로: data\submission.csv

파일 다운로드 완료

데이터셋: 손글씨 이미지 분류
파일경로: data\test.csv

파일 다운로드 완료

데이터셋: 손글씨 이미지 분류
파일경로: data\train.csv



In [5]:
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")
submission = pd.read_csv("data/submission.csv")
submission # 제출 파일

,ImageId,Label
0,1,NaN
1,2,NaN
2,3,NaN
3,4,NaN
4,5,NaN
...,...,...
10495,10496,NaN
10496,10497,NaN
10497,10498,NaN
10498,10499,NaN


### 결과 파일 제출 테스트

In [54]:
# 평가때는 아래 과정 없음
submission

,ImageId,Label
0,1,NaN
1,2,NaN
2,3,NaN
3,4,NaN
4,5,NaN
...,...,...
10495,10496,NaN
10496,10497,NaN
10497,10498,NaN
10498,10499,NaN


In [14]:
pjt.submit(submission)

파일을 저장하였습니다. 파일명: submission-09-21-45.csv
제출 여부 :success
오늘 제출 횟수 : 1
제출 결과:0


## 여기서부터 코드를 작성/수정하세요
#### - 데이터셋 로드, 전처리(필요시), 모델생성/컴파일, 학습
#### - 아래 샘플코드들은 실제 시험 시 없을수 있습니다

In [6]:
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")
submission = pd.read_csv("data/submission.csv")

In [7]:
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
x_train = train.iloc[:,1:]
y_train = train.iloc[:,0]
x_test = test
x_train = np.array(x_train).reshape(-1, 28, 28)
x_test = np.array(x_test).reshape(-1, 28, 28)
x_train.shape, y_train.shape, x_test.shape

((31500, 28, 28), (31500,), (10500, 28, 28))

In [9]:
from sklearn.preprocessing import OneHotEncoder
oh_enc = OneHotEncoder(dtype=int)

oh_enc.fit([[i] for i in range(10)])
y_train = oh_enc.transform(np.array(y_train).reshape(-1, 1)).toarray()

In [10]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

model = Sequential()
model.add(Conv2D(32, kernel_size = (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [11]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.001)
e_st = EarlyStopping(patience = 10, restore_best_weights=True)
history = model.fit(x_train, y_train, epochs=100, validation_split=0.25, callbacks=[e_st, reduce_lr], batch_size=256)

Epoch 1/100
93/93 [==============================] - 8s 78ms/step - loss: 1.6304 - accuracy: 0.7232 - val_loss: 0.3292 - val_accuracy: 0.9034 - lr: 0.0010
Epoch 2/100
93/93 [==============================] - 7s 73ms/step - loss: 0.2399 - accuracy: 0.9284 - val_loss: 0.2044 - val_accuracy: 0.9375 - lr: 0.0010
Epoch 3/100
93/93 [==============================] - 7s 73ms/step - loss: 0.1545 - accuracy: 0.9552 - val_loss: 0.1785 - val_accuracy: 0.9470 - lr: 0.0010
Epoch 4/100
93/93 [==============================] - 7s 72ms/step - loss: 0.1158 - accuracy: 0.9648 - val_loss: 0.1442 - val_accuracy: 0.9595 - lr: 0.0010
Epoch 5/100
93/93 [==============================] - 7s 73ms/step - loss: 0.0837 - accuracy: 0.9735 - val_loss: 0.1231 - val_accuracy: 0.9627 - lr: 0.0010
Epoch 6/100
93/93 [==============================] - 6s 69ms/step - loss: 0.0684 - accuracy: 0.9778 - val_loss: 0.1124 - val_accuracy: 0.9652 - lr: 0.0010
Epoch 7/100
93/93 [==============================] - 6s 68ms/step - lo

### 생성한 정답의 shape 및 타입 확인

In [12]:
# 아래의 코드는 필요에따라 수정해서 사용합니다
pred = model.predict(x_test)

329/329 [==============================] - 3s 8ms/step


In [13]:
your_answer = [np.argmax(row) for row in pred]
your_answer[:10]

[9, 9, 3, 7, 0, 8, 2, 5, 4, 0]

In [37]:
submission['Label'] = your_answer
submission

,ImageId,Label
0,1,9
1,2,9
2,3,3
3,4,7
4,5,0
...,...,...
10495,10496,3
10496,10497,2
10497,10498,0
10498,10499,6


## 결과 제출 (수정하지 마세요)

### 순서대로 평가가 진행됩니다. 실행은 1회만 해주세요.

- 느리다고 중지 후 다시 평가 코드를 실행하는 경우 제출 과정에서 패널티가 발생할 수 있습니다. (제출 횟수 이슈 발생 가능)

- 전체 점수는 [평가시스템](http://manage.jaen.kr)에서 확인할 수 있습니다.
    - 프로젝트 > 손글씨 이미지분류 > 리더보드 > 같은 반 리더보드

- 아이디: 본인 이메일 아이디

In [38]:
pjt.submit(submission)

파일을 저장하였습니다. 파일명: submission-09-47-59.csv
제출 여부 :success
오늘 제출 횟수 : 3
제출 결과:0.974
